In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
df = pd.read_csv('../data/nbre_equipement_parcommuneetcodepostal_ET_LatitudeLongitude_ET_POPULATION.csv', sep="\t")

In [3]:
df.head()

,Commune,PTOT,REG,DEP,Région,latitude,longitude,code_insee,Type équipement ou lieu,count,categorie
0,L'Abergement-Clémenciat,848.0,84.0,01,Auvergne-Rhône-Alpes,NaN,NaN,01001,Monument,1,patrimoine
1,Ambérieu-en-Bugey,15240.0,84.0,01,Auvergne-Rhône-Alpes,NaN,NaN,01004,Bibliothèque,1,vivant
2,Ambérieu-en-Bugey,15240.0,84.0,01,Auvergne-Rhône-Alpes,NaN,NaN,01004,Cinéma,1,vivant
3,Ambérieu-en-Bugey,15240.0,84.0,01,Auvergne-Rhône-Alpes,NaN,NaN,01004,Monument,3,patrimoine
4,Ambérieu-en-Bugey,15240.0,84.0,01,Auvergne-Rhône-Alpes,NaN,NaN,01004,Service d'archives,1,patrimoine


In [4]:
#type of the data in columns

df.dtypes

Commune                     object
PTOT                       float64
REG                        float64
DEP                         object
Région                      object
latitude                   float64
longitude                  float64
code_insee                  object
Type équipement ou lieu     object
count                        int64
categorie                   object
dtype: object

In [5]:
df.head()

,Commune,PTOT,REG,DEP,Région,latitude,longitude,code_insee,Type équipement ou lieu,count,categorie
0,L'Abergement-Clémenciat,848.0,84.0,01,Auvergne-Rhône-Alpes,NaN,NaN,01001,Monument,1,patrimoine
1,Ambérieu-en-Bugey,15240.0,84.0,01,Auvergne-Rhône-Alpes,NaN,NaN,01004,Bibliothèque,1,vivant
2,Ambérieu-en-Bugey,15240.0,84.0,01,Auvergne-Rhône-Alpes,NaN,NaN,01004,Cinéma,1,vivant
3,Ambérieu-en-Bugey,15240.0,84.0,01,Auvergne-Rhône-Alpes,NaN,NaN,01004,Monument,3,patrimoine
4,Ambérieu-en-Bugey,15240.0,84.0,01,Auvergne-Rhône-Alpes,NaN,NaN,01004,Service d'archives,1,patrimoine


In [6]:
# filter df by Nom commune principale = 80485.0

df_filtered= df[df['code_insee'] == '13056']

In [7]:
df_filtered['Commune'].iloc[0]

'Martigues'

In [8]:
fig = px.sunburst(
    data_frame=df_filtered,
    path=["categorie", "Type équipement ou lieu"],
    values="count",
    title=f"<b>Commune:</b> {df_filtered['Commune'].iloc[0]}",
    color="Type équipement ou lieu",  
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig.update_traces(
    texttemplate="%{label}<br>%{value:,}"
)

population_number = int(df_filtered['PTOT'].iloc[0]) 
fig.add_annotation(
    text=f"<b>Population:</b>{population_number:,}",  
    x=0.5,  
    y=1.10,  
    showarrow=False,
    font=dict(size=22, color="black", family="Arial"), 
    align="center", 
    xref="paper", 
    yref="paper"  
)


fig.update_layout(
    margin=dict(t=100, l=0, r=0, b=0),
    title_font=dict(
        family="Arial, sans-serif",  
        size=22,  
        color="black"  
    ),
    font=dict(
        family="Arial, sans-serif",  
        size=14,  
        color="black" 
    )
)

fig.show()


In [9]:
fig = px.bar(df_filtered.sort_values(by=['count', 'categorie'], ascending=[False, True]), 
             x='Type équipement ou lieu', 
             y='count', 
             color='categorie', 
             title="Equipment Type Distribution by Category",
             labels={'count': 'Event Count', 'Type équipement ou lieu': 'Equipment Type'},
             color_discrete_sequence=px.colors.qualitative.Pastel)

fig.update_traces(text=df_filtered.sort_values(by=['count', 'categorie'], ascending=[False, True])['count'], textposition='inside', insidetextanchor='middle')


fig.show()


In [10]:
total_count = df_filtered['count'].sum()

In [11]:
# Sunburst Chart
fig_sunburst = px.sunburst(
    data_frame=df_filtered,
    path=["categorie", "Type équipement ou lieu"],
    values=(df_filtered['count'] / df_filtered['count'].sum()) * 100,
    color="Type équipement ou lieu",  
    color_discrete_sequence=px.colors.qualitative.Pastel,
    hover_data={'count': False}
)

fig_sunburst.update_traces(
    texttemplate="%{label}<br>%{value:,.1f}%"

)

# Bar Chart
fig_bar = px.bar(df_filtered.sort_values(by=['count', 'categorie'], ascending=[False, True]), 
                 x='Type équipement ou lieu', 
                 y='count', 
                 color='categorie', 
                 labels={'count': 'Event Count', 'Type équipement ou lieu': 'Equipment Type'},
                 color_discrete_sequence=px.colors.qualitative.Pastel)

fig_bar.update_traces(text=df_filtered.sort_values(by=['count', 'categorie'], ascending=[False, True])['count'], 
                      textposition='outside', insidetextfont=dict(size=20))

# Create a subplot with two rows and one column
fig = make_subplots(
    rows=2, cols=1, 
    row_heights=[0.7, 0.3],  # Adjust size ratio for sunburst and bar chart
    shared_xaxes=True,
    vertical_spacing=0.1,
    specs=[[{"type": "sunburst"}], [{"type": "bar"}]]
)

# Add sunburst and bar chart traces to the subplot
for trace in fig_sunburst['data']:
    fig.add_trace(trace, row=1, col=1)

for trace in fig_bar['data']:
    fig.add_trace(trace, row=2, col=1)

# Add Commune title annotation
commune_title = f"<b>Commune:</b> {df_filtered['Commune'].iloc[0]}"
fig.add_annotation(
    text=commune_title,  
    x=0.5,  
    y=1.15,  
    showarrow=False,
    font=dict(size=22, color="black", family="Arial"), 
    align="center", 
    xref="paper", 
    yref="paper"  
)

# Add Population annotation below the Commune title
population_number = int(df_filtered['PTOT'].iloc[0]) 
fig.add_annotation(
    text=f"<b>Population:</b> {population_number:,}",  
    x=0.5,  
    y=1.09,  
    showarrow=False,
    font=dict(size=22, color="black", family="Arial"), 
    align="center", 
    xref="paper", 
    yref="paper"  
)

# Update layout for the entire figure
fig.update_layout(
    margin=dict(t=80, l=50, r=50, b=50),
    title_font=dict(
        family="Arial, sans-serif",  
        size=22,  
        color="black"  
    ),
    font=dict(
        family="Arial, sans-serif",  
        size=14,  
        color="black" 
    ),
    height=600,
    width=500, 
    showlegend=True, 
    legend=dict(
        orientation="h",  
        yanchor="top",  
        y=-0.2,  
        xanchor="center", 
        x=0.5 
    )
)

fig.show()


In [12]:
total_count = df_filtered['count'].sum()

In [13]:
total_count

np.int64(26)

In [14]:
fig = px.treemap(df_filtered, 
                 path=['categorie', 'Type équipement ou lieu'], 
                 values='count', 
                 labels={'count': 'Event Count'},
                 color='count',
                 color_continuous_scale='Purp',
                 hover_data={'count': True})

fig.update_traces(textinfo="label+value", textfont_size=18)

fig.update_layout(
    coloraxis_colorbar=dict(
        title="Event Count",
        thicknessmode="pixels", thickness=20,
        lenmode="pixels", len=300,
        yanchor="top", y=1,
        ticks="outside", ticksuffix=" events",
        dtick=5
    )
)

fig.show()

In [15]:
import plotly.express as px
from plotly.subplots import make_subplots

# Sunburst Chart
fig_sunburst = px.sunburst(
    data_frame=df_filtered,
    path=["categorie", "Type équipement ou lieu"],
    values=(df_filtered['count'] / df_filtered['count'].sum()) * 100,
    color="Type équipement ou lieu",  
    color_discrete_sequence=px.colors.qualitative.Pastel,
    hover_data={'count': False}
)

fig_sunburst.update_traces(
    texttemplate="%{label}<br>%{value:,.1f}%"
)

# Treemap Chart
fig_treemap = px.treemap(df_filtered, 
                         path=['categorie', 'Type équipement ou lieu'], 
                         values='count', 
                         labels={'count': 'Event Count'},
                         color='count',
                         color_continuous_scale='Purp',
                         hover_data={'count': True})

fig_treemap.update_traces(textinfo="label+value", textfont_size=18)

# Create a subplot with two rows and one column
fig = make_subplots(
    rows=2, cols=1, 
    row_heights=[0.7, 0.3],  # Adjust size ratio for sunburst and treemap
    shared_xaxes=True,
    vertical_spacing=0.1,
    specs=[[{"type": "sunburst"}], [{"type": "treemap"}]]
)

# Add sunburst and treemap chart traces to the subplot
for trace in fig_sunburst['data']:
    fig.add_trace(trace, row=1, col=1)

for trace in fig_treemap['data']:
    fig.add_trace(trace, row=2, col=1)

# Add Commune title annotation
commune_title = f"<b>Commune:</b> {df_filtered['Commune'].iloc[0]}"
fig.add_annotation(
    text=commune_title,  
    x=0.5,  
    y=1.15,  
    showarrow=False,
    font=dict(size=22, color="black", family="Arial"), 
    align="center", 
    xref="paper", 
    yref="paper"  
)

# Add Population annotation below the Commune title
population_number = int(df_filtered['PTOT'].iloc[0]) 
fig.add_annotation(
    text=f"<b>Population:</b> {population_number:,}",  
    x=0.5,  
    y=1.09,  
    showarrow=False,
    font=dict(size=22, color="black", family="Arial"), 
    align="center", 
    xref="paper", 
    yref="paper"  
)

# Update layout for the entire figure
fig.update_layout(
    margin=dict(t=80, l=50, r=50, b=50),
    title_font=dict(
        family="Arial, sans-serif",  
        size=22,  
        color="black"  
    ),
    font=dict(
        family="Arial, sans-serif",  
        size=14,  
        color="black" 
    ),
    height=800,  # Adjust height for two charts
    width=500, 
    showlegend=True, 
    legend=dict(
        orientation="h",  
        yanchor="top",  
        y=-0.2,  
        xanchor="center", 
        x=0.5 
    ),
    coloraxis_colorbar=dict(
        title="Event Count",
        thicknessmode="pixels", thickness=20,
        lenmode="pixels", len=300,
        yanchor="top", y=1,
        ticks="outside", ticksuffix=" events",
        dtick=5
    )
)

# Show the figure with both sunburst and treemap charts
fig.show()
